In [1]:
# 导入必要的模块
import json

from src.HandPoseRecorder import HandPoseRecordPool, HandPoseRecorder
from src.guitar.Guitar import Guitar
from src.guitar.GuitarString import createGuitarStrings
from src.hand.LeftFinger import LeftFinger
from src.hand.LeftHand import LeftHand
from src.midi.midiToNote import calculate_frame, get_tempo_changes, midiToGuitarNotes
from src.midi.midiToNote import export_midi_info

# 定义参数（可以根据需要修改）
avatar = '户山香澄_E'
midi_name = "户山香澄"
track_number = [2,3]

midiFilePath = f"asset/midi/{midi_name}.mid"
channel_number = -1  
FPS = 24
guitar_string_notes = ["e", "b", "G", "D", "A", "E1"] # guitar
# guitar_string_notes = ["G1", "D1", "A1", "E2"]  # bass
octave_down_checkbox = False
capo_number = 0
disable_barre = False # 是否禁用横按

# 第一步：初始化文件路径和基本参数
filename = midiFilePath.split("/")[-1].split(".")[0]
track_number_string = "_".join([str(i) for i in track_number])
notes_map_file = f"output/midi_info/{filename}_{track_number_string}_notes_map.json"
messages_file = f"output/midi_info/{filename}_{track_number_string}_messages.json"
left_hand_recorder_file = f"output/hand_recorder/{filename}_{track_number_string}_lefthand_recorder.json"
left_hand_animation_file = f"output/hand_animation/{avatar}_{filename}_{track_number_string}_lefthand_animation.json"
right_hand_recorder_file = f"output/hand_recorder/{filename}_{track_number_string}_righthand_recorder.json"
right_hand_animation_file = f"output/hand_animation/{avatar}_{filename}_{track_number_string}_righthand_animation.json"
guitar_string_recorder_file = f"output/string_recorder/{filename}_{track_number_string}_guitar_string_recorder.json"

print("文件路径已初始化:")
print(f"  音符映射文件: {notes_map_file}")
print(f"  消息文件: {messages_file}")
print(f"  左手记录文件: {left_hand_recorder_file}")
print(f"  左手动画文件: {left_hand_animation_file}")
print(f"  右手记录文件: {right_hand_recorder_file}")
print(f"  右手动画文件: {right_hand_animation_file}")
print(f"  吉他弦动画文件: {guitar_string_recorder_file}")

# 第二步：解析MIDI文件获取音符和速度变化信息
tempo_changes, ticks_per_beat = get_tempo_changes(midiFilePath)
notes_map, pitch_wheel_map, messages = midiToGuitarNotes(
    midiFilePath, useTracks=track_number, useChannel=channel_number,
    octave_down_checkbox=octave_down_checkbox, capo_number=capo_number)

# 保存notes_map和messages到文件
with open(notes_map_file, "w") as f:
    json.dump(notes_map, f, indent=4)
with open(messages_file, "w") as f:
    json.dump(messages, f, indent=4)

midi_info = export_midi_info(midi_name)
print(midi_info)

print(f'全曲的速度变化是:')
for track, tempo, tick in tempo_changes:
    print(f'在{track}轨，tick为{tick}时，速度变为{tempo}')

print(f'\n全曲的每拍tick数是:{ticks_per_beat}\n')

# 计算总时长
total_tick = notes_map[-1]['real_tick']
total_frame = calculate_frame(tempo_changes, ticks_per_beat, FPS, total_tick)
total_time = total_frame/FPS
print(f'如果以{FPS}的fps做成动画，一共是{total_tick} ticks, 合计{total_frame}帧, 约{total_time}秒')

文件路径已初始化:
  音符映射文件: output/midi_info/户山香澄_2_3_notes_map.json
  消息文件: output/midi_info/户山香澄_2_3_messages.json
  左手记录文件: output/hand_recorder/户山香澄_2_3_lefthand_recorder.json
  左手动画文件: output/hand_animation/户山香澄_E_户山香澄_2_3_lefthand_animation.json
  右手记录文件: output/hand_recorder/户山香澄_2_3_righthand_recorder.json
  右手动画文件: output/hand_animation/户山香澄_E_户山香澄_2_3_righthand_animation.json
  吉他弦动画文件: output/string_recorder/户山香澄_2_3_guitar_string_recorder.json
Track 0: 
Track 1: 
Track 2: çµå­åä»
Track 2, Channel 0: Overdriven Guitar
Track 2, Channel 0: Overdriven Guitar
Track 2, Channel 0: Overdriven Guitar
Track 2, Channel 0: Overdriven Guitar
Track 2, Channel 0: Overdriven Guitar
Track 3: çµå­åä»
Track 3, Channel 0: Acoustic Grand Piano
Track 3, Channel 0: Acoustic Grand Piano
Track 3, Channel 0: Acoustic Grand Piano
Track 3, Channel 0: Acoustic Grand Piano
Track 3, Channel 0: Acoustic Grand Piano

全曲的速度变化是:
在1轨，tick为0时，速度变为309278

全曲的每拍tick数是:96

如果以24的fps做成动画，一共是7968 ticks, 合计616.081

In [2]:
# 第三步：初始化吉他和左手
guitar_string_list = createGuitarStrings(guitar_string_notes)
max_string_index = len(guitar_string_list) - 1

# 输出guitar_string_list
for i, string in enumerate(guitar_string_list):
    print(f"弦{i}的音符范围是: {string.getBaseNote()} ~ {string.getBaseNote() + 12}")

# 初始化吉他
guitar = Guitar(guitar_string_list)

# 设定各手指初始状态
leftFingers = [
    LeftFinger(1, guitar_string_list[1], 5),
    LeftFinger(2, guitar_string_list[1], 6),
    LeftFinger(3, guitar_string_list[1], 7),
    LeftFinger(4, guitar_string_list[1], 8)
]

# 初始化左手
initLeftHand = LeftHand(leftFingers)

# 初始化第一个记录器
handPoseRecord = HandPoseRecorder()
handPoseRecord.addHandPose(initLeftHand, 0, 0)

# 初始化记录池
handPoseRecordPool = HandPoseRecordPool(100)
handPoseRecordPool.insert_new_hand_pose_recorder(handPoseRecord, 0)

print("吉他和左手初始化完成")
print(f"最大弦索引: {max_string_index}")
print(
    f"初始手指位置: {[f'手指{f._fingerIndex}在{f.stringIndex}弦{f.fret}品' for f in leftFingers]}")

弦0的音符范围是: 64 ~ 76
弦1的音符范围是: 59 ~ 71
弦2的音符范围是: 55 ~ 67
弦3的音符范围是: 50 ~ 62
弦4的音符范围是: 45 ~ 57
弦5的音符范围是: 40 ~ 52
吉他和左手初始化完成
最大弦索引: 5
初始手指位置: ['手指1在1弦5品', '手指2在1弦6品', '手指3在1弦7品', '手指4在1弦8品']


In [3]:
from FretDaner import update_recorder_pool
# 第四步：生成左手按弦数据
current_recoreder_num = 0
previous_recoreder_num = 0
total_steps = len(notes_map)

print('开始生成左手按弦数据')
update_recorder_pool(total_steps, guitar, handPoseRecordPool, notes_map, current_recoreder_num, previous_recoreder_num)

# 获取最优解
bestHandPoseRecord = handPoseRecordPool.curHandPoseRecordPool[0]
bestEntropy = bestHandPoseRecord.currentEntropy
print(f"最小消耗熵为：{bestEntropy}")
bestHandPoseRecord.save(left_hand_recorder_file, tempo_changes, ticks_per_beat, FPS)
print(f"总音符数应该为{total_steps}")
print(f"实际输出音符数为{len(bestHandPoseRecord.handPoseList)}")

开始生成左手按弦数据


Processing: 100%|█████████████████████████████████████████████████| 87/87 [00:29<00:00,  2.97step/s]

最小消耗熵为：749.5755231017476
去重统计: 原始记录 87 条，去重后 87 条，删除重复记录 0 条
总音符数应该为87
实际输出音符数为88


In [4]:
from src.animate.animate import addPitchwheel
from src.midi.midiToNote import calculate_frame
# 第五步：处理推弦动作（如果有）
if len(pitch_wheel_map) > 0:
    print("检测到推弦动作，正在处理...")
    for item in pitch_wheel_map:
        frame = calculate_frame(tempo_changes, ticks_per_beat, FPS, item['real_tick'])
        item['frame'] = frame
    addPitchwheel(left_hand_recorder_file, pitch_wheel_map)
    print(f"已处理{len(pitch_wheel_map)}个推弦动作")
else:
    print("未检测到推弦动作")

未检测到推弦动作


In [5]:
from src.animate.animate import leftHand2Animation
# 第六步：生成左手动画数据
print("正在生成左手动画数据...")
leftHand2Animation(avatar, left_hand_recorder_file,
                   left_hand_animation_file, FPS, max_string_index,disable_barre=disable_barre)
print("左手动画数据生成完成")

正在生成左手动画数据...
左手动画数据生成完成


In [6]:
from src.HandPoseRecorder import HandPoseRecordPool,RightHandRecorder
from src.animate.animate import ElectronicRightHand2Animation
from FretDaner import update_right_hand_recorder_pool, leftHand2ElectronicRightHand
from src.hand.RightHand import RightHand

# 第七步：生成右手演奏数据
print('开始生成右手演奏数据')
if avatar.endswith("_E"):
    print("检测到电吉他角色，使用电吉他右手处理...")
    leftHand2ElectronicRightHand(
        left_hand_recorder_file, right_hand_recorder_file)
    ElectronicRightHand2Animation(
        avatar, right_hand_recorder_file, right_hand_animation_file, FPS, max_string_index)
    print("电吉他右手动画数据生成完成")
else:
    is_playing_bass = max_string_index < 5
    print("使用古典吉他/finger bass右手处理...")
    initRightHand = RightHand(
        usedFingers=[], rightFingerPositions=[max_string_index, 2, 1, 0], preUsedFingers=[],is_playing_bass=is_playing_bass)

    initRightHandRecorder = RightHandRecorder()
    initRightHandRecorder.addHandPose(initRightHand, 0, 0)

    rightHandRecordPool = HandPoseRecordPool(100)
    rightHandRecordPool.insert_new_hand_pose_recorder(
        initRightHandRecorder, 0)

    current_recoreder_num = 0
    previous_recoreder_num = current_recoreder_num

    update_right_hand_recorder_pool(
        left_hand_recorder_file, rightHandRecordPool, current_recoreder_num, previous_recoreder_num, max_string_index)

    # 获取最优解
    bestHandPoseRecord = rightHandRecordPool.curHandPoseRecordPool[0]
    bestEntropy = bestHandPoseRecord.currentEntropy
    print(f"最小消耗熵为：{bestEntropy}")
    bestHandPoseRecord.save(right_hand_recorder_file,
                            tempo_changes, ticks_per_beat, FPS)

开始生成右手演奏数据
检测到电吉他角色，使用电吉他右手处理...


Processing: 100%|█████████████████████████████████████████████| 87/87 [00:00<00:00, 167541.07step/s]


电吉他右手动画数据生成完成


In [7]:
from src.animate.animate import rightHand2Animation
# 第八步：生成古典吉他右手动画数据
print('开始生成右手演奏数据')
if not avatar.endswith("_E"):        

    rightHand2Animation(avatar, right_hand_recorder_file,
                        right_hand_animation_file, FPS, max_string_index)
    print("古典吉他右手动画数据生成完成")

开始生成右手演奏数据


In [8]:
from src.animate.animate import animated_guitar_string
# 第九步：生成吉他弦动画数据
print('开始生成吉他弦动画数据...')
animated_guitar_string(left_hand_recorder_file,
                       guitar_string_recorder_file, FPS)
print("吉他弦动画数据生成完成")

开始生成吉他弦动画数据...
吉他弦动画数据生成完成


In [9]:
# 最终结果输出
finall_info = f'''全部执行完毕:
recorder文件被保存到了:
  - {left_hand_recorder_file}
  - {right_hand_recorder_file}
动画文件被保存到了:
  - {left_hand_animation_file}
  - {right_hand_animation_file}
吉它弦动画文件被保存到了:
  - {guitar_string_recorder_file}'''

print(finall_info)

全部执行完毕:
recorder文件被保存到了:
  - output/hand_recorder/户山香澄_2_3_lefthand_recorder.json
  - output/hand_recorder/户山香澄_2_3_righthand_recorder.json
动画文件被保存到了:
  - output/hand_animation/户山香澄_E_户山香澄_2_3_lefthand_animation.json
  - output/hand_animation/户山香澄_E_户山香澄_2_3_righthand_animation.json
吉它弦动画文件被保存到了:
  - output/string_recorder/户山香澄_2_3_guitar_string_recorder.json
